<a href="https://colab.research.google.com/github/GitHub-Bong/Toxic-Comment-Challenge/blob/master/0408_maintain_stopwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount, Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
import sys, os, re, string, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, concatenate, add
from tensorflow.keras.layers import Conv1D, GRU, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

# Load Data

In [ ]:
train = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/train.csv/train.csv')
test = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/test.csv/test.csv')

In [ ]:
test['preprocessed'] = test['comment_text']

In [ ]:
train_fr = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/Ho-colab/train_fr.csv')
train_de = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/Ho-colab/train_de.csv')
train_es = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/Ho-colab/train_es.csv')

In [ ]:
train_augumented = pd.concat([train, train_fr, train_de, train_es])

from sklearn.utils import shuffle
train_augumented = shuffle(train_augumented)

In [ ]:
train_augumented.head(4)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
157793,e36fd259742139d4,"""\n\nRe: IRC\n\nWhere the hell did you get thi...",0,0,0,0,0,0
25030,42333ab636e5a385,Someone who is too incompetent to be stationed...,0,0,1,0,0,0
105858,364e2028970c5df0,Review of GA\n: This review is transcribed fro...,0,0,0,0,0,0
111204,52ea2868a140e509,"""\n\nThis was far from """"vandalism"""" if thats ...",0,0,0,0,0,0


In [ ]:
train_augumented.shape

(638284, 8)

In [ ]:
train_augumented['preprocessed'] = train_augumented['comment_text']

# Preprocessing

In [ ]:
# 1. Capitalization / Lower case'
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: x.lower())
test['preprocessed'] = test['preprocessed'].apply(lambda x: x.lower())
# 2. Remove ~.jpg , (UTC)
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: re.sub(r"\S+\.jpg|\(UTC\)", " ", x))
test['preprocessed'] = test['preprocessed'].apply(lambda x: re.sub(r"\S+\.jpg|\(UTC\)", " ", x))
# 3. Expand the Contractions
!pip install contractions
import contractions
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: contractions.fix(x))
test['preprocessed'] = test['preprocessed'].apply(lambda x: contractions.fix(x))
# 4. Remove URLs
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: re.sub(r"https?://\S+|www\.\S+", " ", x))
test['preprocessed'] = test['preprocessed'].apply(lambda x: re.sub(r"https?://\S+|www\.\S+", " ", x))
# 5. Remove HTML tags
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", " ", x))
test['preprocessed'] = test['preprocessed'].apply(lambda x: re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", " ", x))
# 6. Remove Non-ASCI
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: re.sub(r'[^\x00-\x7f]',' ', x))
test['preprocessed'] = test['preprocessed'].apply(lambda x: re.sub(r'[^\x00-\x7f]',' ', x))
# 7. Remove punctuations
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', " ", x))
test['preprocessed'] = test['preprocessed'].apply(lambda x: re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', " ", x))
# 8. Capitalization / Lower case
train_augumented['preprocessed'] = train_augumented['preprocessed'].apply(lambda x: x.lower())
test['preprocessed'] = test['preprocessed'].apply(lambda x: x.lower())
# # Remove stopwords
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# train_fr['2-trans'] = train_fr['2-trans'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

     |████████████████████████████████| 266kB 28.0MB/s 
     |████████████████████████████████| 327kB 55.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85395 sha256=ca3eb22a6c581994df07aa5e0818b17264271e669dcc2f80c17c80888d2290f6
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


## Substitute zero len

### train data

In [ ]:
# Find zero len comment
zerolen_t = list()
num = 0
for i in train_augumented['preprocessed']:
  if not re.search('\S',i):
    zerolen_t.append(num)
  num += 1
print(len(zerolen_t))  # 전처리 되어서 ' ' 형태된 갯수 

66


In [ ]:
pd.set_option('max_colwidth', 200)
pd.set_option("max_rows", None)
train_augumented.iloc[zerolen_t,1:]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocessed
151221,"""::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n""",0,0,0,0,0,0,useless
62937,"""::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\n""",0,0,0,0,0,0,useless
76037,http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html,0,0,0,0,0,0,useless
153616,"http://en.wikipedia.org/w/index.php?title=Crash_Bandicoot_%28series%29&diff;=prev&oldid;=484470162]], [], [], [], [] [[",0,0,0,0,0,0,useless
68469,http://en.wikipedia.org/wiki/Towns_in_the_Former_RSK\nhttp://en.wikipedia.org/wiki/Wikipedia:Articles_for_deletion/Geography_of_the_Former_Republic_of_Serbian_Krajina ( ).,0,0,0,0,0,0,useless
109944,http://www.newswire.ca/fr/story/358197/domaine-pinnacle-ice-cider-now-available-in-new-european-and-asian-markets.,0,0,0,0,0,0,useless
94038,http://wikipediawehaveaproblem.com/evidence-of-cyberbullying-wp-editors-and-admins/ http://wikipediawehaveaproblem.com/evidence-of-cyberbullying-wp-editors-and-admins/,0,0,0,0,0,0,useless
64904,http://en.wikipedia.org/wiki/Wikipedia_Talk:No_original_research/archive15#YouTube_art_as_primary_source,0,0,0,0,0,0,useless
147665,http://www.world-pyramids.com/atr/usa/monkmound.html\nhttp://www.freewebs.com/historyofmonksmound/index.htm\nhttp://www.bibliotecapleyades.net/arqueologia/monks_mound.htm,0,0,0,0,0,0,useless
41648,[]\nhttp://www.smallholder.co.uk/news/878983.honey_storage/,0,0,0,0,0,0,useless


In [ ]:
train_augumented.iloc[zerolen_t,8] = 'useless'

In [ ]:
train_augumented.iloc[zerolen_t,[1,8]].head(5)

,comment_text,preprocessed
151221,"""::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n""",useless
62937,"""::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\n""",useless
76037,http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html,useless
153616,"http://en.wikipedia.org/w/index.php?title=Crash_Bandicoot_%28series%29&diff;=prev&oldid;=484470162]], [], [], [], [] [[",useless
68469,http://en.wikipedia.org/wiki/Towns_in_the_Former_RSK\nhttp://en.wikipedia.org/wiki/Wikipedia:Articles_for_deletion/Geography_of_the_Former_Republic_of_Serbian_Krajina ( ).,useless


In [ ]:
train_augumented.to_csv("/content/drive/Shareddrives/SOGANG Parrot/All-submission/0408-contain-stopwords-augu-train.csv", index=False)

In [ ]:
train_augumented = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/All-submission/0408-contain-stopwords-augu-train.csv')
train_augumented.tail(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocessed
638281,c1883e3d2dc0bce7,"""\n\nPlease stop. If you continue deleting or ...",0,0,0,0,0,0,\n\nplease stop if you continue deleting or ...
638282,b496c784da581dfc,"""\n\nAgain, I hope this website helps. By the ...",0,0,0,0,0,0,\n\nagain i hope this website helps by the ...
638283,1744b108340d0ee3,This is what I found as sources \n Karl F Hi...,0,0,0,0,0,0,this is what i found as sources \n karl f hi...


In [ ]:
# Find zero len comment
zerolen_t = list()
num = 0
for i in train_augumented['preprocessed']:
  if re.search('useless',i):
    zerolen_t.append(num)
  num += 1
print(len(zerolen_t))  # 전처리 되어서 ' ' 형태된 갯수 

3050


------------------

### test data

In [ ]:
# Find zero len comment
zerolen_te = list()
num = 0
for i in test['preprocessed']:
  if not re.search('\S',i):
    zerolen_te.append(num)
  num += 1
print(len(zerolen_te))  # 전처리 되어서 ''된 갯수 

827


In [ ]:
test.iloc[zerolen_te,[1,2]]

,comment_text,preprocessed
34,일이삼사오육칠팔구하고십이요 에헤헤 으헤 으헤 으허허,
114,পরিচয় \n\n নাম-মোঃফরহাদ ইসলাম \n পিতা-মোঃ শাহজাহান মিয়া \n মাতা-মোছাঃফাহিমা বেগম \n গ্রামঃ জানকিপুর সলই মন্ডল পাড়া \n ইউনিয়নঃনিলাখিয়া \n উপজেলাঃবকশীগঞ্জ \n জেলাঃজামালপুর \n বিভাগঃঢাকা \n\n...,\n\n \n \n \n \n \n \n \n \n\n...
119,السلام عليكم و رحمة الله و بركاته الا الجميع \n تفضلوا جميعا,\n
419,Радченко ис мы дирты гаы анал сех славе!!,
506,== ޗޮޮލ! == \n\n މަލދިވެސ ިސ ަ ޕރެތތޔ ގޮޮދ ިސލަނދ ނަތިޮނ. ި ސުރެ އޮުލދ ލޮވެ ތޮ ގޮ ތހެރެ އިތހ މޔ ފރިެނދސ!,\n\n
578,حاول قراءة صفحات المساعدة والتعليمات \n *,\n
582,"Чини ми се да малим википедијама не преостаје ништа друго него да преводи чланке са вечих. Иначе за СХ википедиу предлажем заједнички нови стандард: јекавица, избациванје ије и ћ, кориштинје славе...",...
674,تعتبر ابنوب من اجمل مدن مصر حيث اصبحت الان قطعة من اوروبا حيث المبانى الشامخة ووجود المصالح الحكومية والنوادى وغيرها وذلك بجهود السيد المحافظ نبيل العزبى واذا نظرنا الى ابنوب وجدنا انها مقسمة الى...,...
819,""" \n ::::::אוקיי אז שיניתי את זה לליד בית חנינא,פשוט באופן כללי כרושמים על איזשהו מקום שנמצא במה שקוראים לו היום """"מזרח ירושלים"""" שהוא לא בתוך ירושלים זה כבר כן קשור לפוליטיקה לדעתי,בכל מקרה תודה ...",\n ...
1278,لڑکی اور لڑکے میں جومحبت پیدا ہوتی ہے، ہوسکتاہے اس کی ابتدا ہی غیر شرعی ہو مثلاً ایک دوسرے سے میل جول ، اور خلوت ، کلام اور بات چيت کرنا اور ایک دوسری کی تصاویر کا تبادلہ وغیرہ۔ یہ سب کام حرام اور...,...


In [ ]:
!pip install google_trans_new
from google_trans_new import google_translator

translator = google_translator()

In [ ]:
zerolen_te.remove(55142)

In [ ]:
translator.translate('حينما اكتب لك بالعربية كلامي يبقى وحين اكتب بالأنجليزية يمسح ) لا اعرف لماذا لكن هل تعرف العربية..؟ المهم انا اريد ان نتعاون	')

'When I write to you in Arabic, my words remain and then write English wipe) I do not know why but do you know Arabic ..? Important I want to cooperate '

In [ ]:
test.iloc[zerolen_te,2] = test.iloc[zerolen_te,1].apply(lambda x: translator.translate(x))

In [ ]:
test.iloc[zerolen_te,[1,2]].head(10)

,comment_text,preprocessed
34,일이삼사오육칠팔구하고십이요 에헤헤 으헤 으헤 으허허,"I have a lot of people in the house,"
114,পরিচয় \n\n নাম-মোঃফরহাদ ইসলাম \n পিতা-মোঃ শাহজাহান মিয়া \n মাতা-মোছাঃফাহিমা বেগম \n গ্রামঃ জানকিপুর সলই মন্ডল পাড়া \n ইউনিয়নঃনিলাখিয়া \n উপজেলাঃবকশীগঞ্জ \n জেলাঃজামালপুর \n বিভাগঃঢাকা \n\n...,Identity\n\n Name-Mohammafhorhad Islam\n Father-Mohammad Shahjahan Mia\n Mother-obligeologist Begum\n Village: Janakipur Soli Mandal laid\n UnionNield\n Upazila scholarship\n District --jamalpur\n...
119,السلام عليكم و رحمة الله و بركاته الا الجميع \n تفضلوا جميعا,"Peace, mercy and blessings of God be upon you They all prefer"
419,Радченко ис мы дирты гаы анал сех славе!!,Radchenko IS We are DirTa Gai Anal Sech Glory !!
506,== ޗޮޮލ! == \n\n މަލދިވެސ ިސ ަ ޕރެތތޔ ގޮޮދ ިސލަނދ ނަތިޮނ. ި ސުރެ އޮުލދ ލޮވެ ތޮ ގޮ ތހެރެ އިތހ މޔ ފރިެނދސ!,== ޗޮޮލ! == މަލދިވެސ ިސ ަ ޕރެތތޔ ގޮޮދ ިސލަނދ. ި ސުރެ އޮުލދ ލޮވެ ތޮ ގޮ ތހެރެ އިތހ މޔ ފރިެނދސ!
578,حاول قراءة صفحات المساعدة والتعليمات \n *,Try to read assistance and instructions *
582,"Чини ми се да малим википедијама не преостаје ништа друго него да преводи чланке са вечих. Иначе за СХ википедиу предлажем заједнички нови стандард: јекавица, избациванје ије и ћ, кориштинје славе...","It seems to me that small wikipedics do nothing left but translates articles from the eternal. Otherwise, for SH Wikipedia, I propose a common new standard: Žekavica, ejection of IJE and ć, use of..."
674,تعتبر ابنوب من اجمل مدن مصر حيث اصبحت الان قطعة من اوروبا حيث المبانى الشامخة ووجود المصالح الحكومية والنوادى وغيرها وذلك بجهود السيد المحافظ نبيل العزبى واذا نظرنا الى ابنوب وجدنا انها مقسمة الى...,Abbid is one of the most beautiful cities of Egypt.
819,""" \n ::::::אוקיי אז שיניתי את זה לליד בית חנינא,פשוט באופן כללי כרושמים על איזשהו מקום שנמצא במה שקוראים לו היום """"מזרח ירושלים"""" שהוא לא בתוך ירושלים זה כבר כן קשור לפוליטיקה לדעתי,בכל מקרה תודה ...",""" :::: Okay I changed it to Lid Beit Hanina, just in general as there are somewhere in what is now called ""East Jerusalem"" today ""that is not in Jerusalem, it is also related to politics in my op..."
1278,لڑکی اور لڑکے میں جومحبت پیدا ہوتی ہے، ہوسکتاہے اس کی ابتدا ہی غیر شرعی ہو مثلاً ایک دوسرے سے میل جول ، اور خلوت ، کلام اور بات چيت کرنا اور ایک دوسری کی تصاویر کا تبادلہ وغیرہ۔ یہ سب کام حرام اور...,"The girl and the boy who is born, may be the beginning of uncontrollable, such as non-sharih, such as melting, and talking to each other, and discussing one's other photos etc. All these work are ..."
